In [4]:
##### import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from statsmodels.tsa.deterministic import DeterministicProcess

In [5]:
# path to the dataset in Kaggle's notebook
path = '../input/store-sales-time-series-forecasting/'

### 1. Compute Moving Average of Oil Prices

In [6]:
# read oil price
data_oil = pd.read_csv(path + 'oil.csv', parse_dates=['date'], infer_datetime_format=True, index_col='date')

########################################################################################################################
# TODO: compute data_oil['ma_oil'] as the moving average of data_oil['dcoilwtico'] with window size 7
# Hint: check the documentation of .rolling() method of pandas.DataFrame
########################################################################################################################
data_oil['ma_oil'] = data_oil['dcoilwtico'].rolling(window=7).mean() # change 'None' to your answer

# Create continguous moving average of oil prices
calendar = pd.DataFrame(index=pd.date_range('2013-01-01', '2017-08-31'))
calendar.index.names=["date"]
# # ########################################################################################################################
# # # TODO 1: merge two DataFrame instances (data_oil and calendar) such that the merged instances has the same indexes
# # # as calendar.
# # # TODO 2: replace each NaN in data_oil['ma_oil'] by the first non-null value before it.
# # # Hint: check the documentation of .merge() and .fillna() methods of pandas.DataFrame
# # ########################################################################################################################
data_oil['ma_oil'].fillna(method='ffill', inplace=True)
calendar = calendar.merge(data_oil, on="date", how="outer")
calendar['ma_oil'].fillna(method='ffill', inplace=True)
calendar.head(15)

,dcoilwtico,ma_oil
date,,
2013-01-01,NaN,NaN
2013-01-02,93.14,NaN
2013-01-03,92.97,NaN
2013-01-04,93.12,NaN
2013-01-05,NaN,NaN
2013-01-06,NaN,NaN
2013-01-07,93.20,NaN
2013-01-08,93.21,NaN
2013-01-09,93.08,NaN


### 2. Create Workday Feature

In [7]:
########################################################################################################################
# TODO: create a True/False feature calendar['wd'] to indicate whether each date is a workday (Monday-Friday) or not.
# Hint: check documentation of pandas.DatetimeIndex.dayofweek
########################################################################################################################
calendar['wd'] = calendar.index.dayofweek # change 'None' to your answer
calendar['wd'] = calendar['wd'].map({0:True, 1:True, 2:True, 3:True, 4:True, 5:False, 6:False})
calendar.head(15) # display some entries of calendar

,dcoilwtico,ma_oil,wd
date,,,
2013-01-01,NaN,NaN,True
2013-01-02,93.14,NaN,True
2013-01-03,92.97,NaN,True
2013-01-04,93.12,NaN,True
2013-01-05,NaN,NaN,False
2013-01-06,NaN,NaN,False
2013-01-07,93.20,NaN,True
2013-01-08,93.21,NaN,True
2013-01-09,93.08,NaN,True


### 3. Read Train and Test Data

In [8]:
df_train = pd.read_csv(path + 'train.csv',
                       usecols=['store_nbr', 'family', 'date', 'sales'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32'},
                       parse_dates=['date'], infer_datetime_format=True)

df_train.date = df_train.date.dt.to_period('D')
df_train = df_train.set_index(['store_nbr', 'family', 'date']).sort_index()

df_train.head(15) # display some entries of the training data

sales
store_nbr family     date             
1         AUTOMOTIVE 2013-01-01    0.0
                     2013-01-02    2.0
                     2013-01-03    3.0
                     2013-01-04    3.0
                     2013-01-05    5.0
                     2013-01-06    2.0
                     2013-01-07    0.0
                     2013-01-08    2.0
                     2013-01-09    2.0
                     2013-01-10    2.0
                     2013-01-11    3.0
                     2013-01-12    2.0
                     2013-01-13    2.0
                     2013-01-14    2.0
                     2013-01-15    1.0

In [9]:
df_test = pd.read_csv(path + 'test.csv',
                      usecols=['store_nbr', 'family', 'date'],
                      dtype={'store_nbr': 'category', 'family': 'category'},
                      parse_dates=['date'], infer_datetime_format=True)

df_test.date = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['store_nbr', 'family', 'date']).sort_index()

df_test.head(15) # display some entries of the testing data

Empty DataFrame
Columns: []
Index: [(1, AUTOMOTIVE, 2017-08-16), (1, AUTOMOTIVE, 2017-08-17), (1, AUTOMOTIVE, 2017-08-18), (1, AUTOMOTIVE, 2017-08-19), (1, AUTOMOTIVE, 2017-08-20), (1, AUTOMOTIVE, 2017-08-21), (1, AUTOMOTIVE, 2017-08-22), (1, AUTOMOTIVE, 2017-08-23), (1, AUTOMOTIVE, 2017-08-24), (1, AUTOMOTIVE, 2017-08-25), (1, AUTOMOTIVE, 2017-08-26), (1, AUTOMOTIVE, 2017-08-27), (1, AUTOMOTIVE, 2017-08-28), (1, AUTOMOTIVE, 2017-08-29), (1, AUTOMOTIVE, 2017-08-30)]

In [10]:
# set the range of data used in training
sdate = '2017-04-01'
edate = '2017-08-15'

# we will train a model that takes feature of a date as input and predicts the sales for each store and family of goods on that date.
y = df_train.unstack(['store_nbr', 'family']).loc[sdate:edate]

# ########################################################################################################################
# # TODO: create the trend feature X: the value for sdate is 1, the value for the next day of sdate is 2, etc.
# # Hint: check the documentation of DeterministicProcess, or this tutorial: https://www.kaggle.com/code/ryanholbrook/trend.
# ########################################################################################################################
from statsmodels.tsa.deterministic import DeterministicProcess

dp = DeterministicProcess(
    index=y.index,  # dates from the training data
    order=1,        # the time dummy (trend)
    drop=True,      # drop terms if necessary to avoid collinearity
)

X = dp.in_sample()

# Extensions
X['oil']  = calendar.loc[sdate:edate]['ma_oil'].values
X['wd']   = calendar.loc[sdate:edate]['wd'].values
X

,trend,oil,wd
date,,,
2017-04-01,1.0,48.570000,False
2017-04-02,2.0,48.570000,False
2017-04-03,3.0,49.034286,True
2017-04-04,4.0,49.561429,True
2017-04-05,5.0,50.150000,True
...,...,...,...
2017-08-11,133.0,49.140000,True
2017-08-12,134.0,49.140000,False
2017-08-13,135.0,49.140000,False


### 4. Train Model!

In [11]:
model = LinearRegression()
model.fit(X, y)
y_pred = pd.DataFrame(model.predict(X), index=X.index, columns=y.columns)

In [12]:
# Results on the training set
y_pred   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()

y_target['sales_pred'] = y_pred['sales'].clip(0.) # Sales should be >= 0

########################################################################################################################
# TODO: show the training loss for each type of product.
# Hint: check the documentation of DataFrame.groupby() and GroupBy.apply().
########################################################################################################################
y_target['train_diff'] = y_target['sales'] - y_target['sales_pred']
y_target.groupby('family').train_diff.apply(lambda x:x.abs().sum())

family
AUTOMOTIVE                    2.111925e+04
BABY CARE                     1.743297e+03
BEAUTY                        1.662243e+04
BEVERAGES                     4.260417e+06
BOOKS                         7.725450e+02
BREAD/BAKERY                  5.827882e+05
CELEBRATION                   4.049035e+04
CLEANING                      2.034643e+06
DAIRY                         9.858472e+05
DELI                          3.978347e+05
EGGS                          3.847354e+05
FROZEN FOODS                  2.317231e+05
GROCERY I                     5.326956e+06
GROCERY II                    7.063961e+04
HARDWARE                      7.883162e+03
HOME AND KITCHEN I            8.419063e+04
HOME AND KITCHEN II           7.543218e+04
HOME APPLIANCES               4.125852e+03
HOME CARE                     4.023826e+05
LADIESWEAR                    3.028118e+04
LAWN AND GARDEN               4.879019e+04
LINGERIE                      2.567281e+04
LIQUOR,WINE,BEER              3.178458e+05
MAGA

In [13]:
# Discussion for 5.4: Based on the results above from the training error per family of product, the model seems to have the most 
# difficulty with classifying the family of GROCERY I, since it has the largest training error.

In [16]:
# Test predictions

stest = '2017-08-16'
etest = '2017-08-31'

########################################################################################################################
# TODO: create the feature matrix of test data.
# Hint: check the documentation of DeterministicProcess.
########################################################################################################################
X_test = dp.out_of_sample(steps=16)
X_test['oil']  = calendar.loc[stest:etest]['ma_oil'].values
X_test['wd']   = calendar.loc[stest:etest]['wd'].values

sales_pred = pd.DataFrame(model.predict(X_test), index=X_test.index, columns=y.columns)
sales_pred = sales_pred.stack(['store_nbr', 'family'])

sales_pred[sales_pred < 0] = 0. # Sales should be >= 0
sales_pred

sales
           store_nbr family                                 
2017-08-16 1         AUTOMOTIVE                     4.866719
                     BABY CARE                      0.000000
                     BEAUTY                         3.613312
                     BEVERAGES                   2198.244983
                     BOOKS                          0.158089
...                                                      ...
2017-08-31 9         POULTRY                      378.200685
                     PREPARED FOODS                91.334172
                     PRODUCE                     1472.628930
                     SCHOOL AND OFFICE SUPPLIES    99.010022
                     SEAFOOD                       14.535122

[28512 rows x 1 columns]

In [15]:
# Create submission

df_sub = pd.read_csv(path + 'sample_submission.csv', index_col='id')
df_sub.sales = sales_pred.values
df_sub.to_csv('submission.csv', index=True)